In [1]:
import getpass
import os
from pathlib import Path
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [2]:
from agno.models.huggingface import HuggingFace
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.models.huggingface import HuggingFace
from agno.models.groq import Groq
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.csv_toolkit import CsvTools


* LLM

In [3]:
hf_model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)


ollama_model=Ollama(id="llama3.2:3b")

groq_model=Groq(id="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))

In [4]:
ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

### Web Searcher Agent

In [5]:
web_searcher = Agent(model=groq_model, 
                     tools=[DuckDuckGoTools()], 
                     name="Web Searcher",  
                     role="Searches the web for information on a topic", 
                     show_tool_calls=True,
                     markdown=False)

response = web_searcher.run("Who is Thomas Edison?")
print(response.content) #latencia 1min 50

 - Running: duckduckgo_search(query=Thomas Edison, max_results=5)

Based on the search results from DuckDuckGo, Thomas Edison was an American inventor and businessman who developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. He is best known for his invention of the phonograph and the incandescent electric light, but he took out 1,093 patents in a variety of fields. Edison's inventions have had a widespread impact on society and have changed the way people live and communicate.


###  RAG Agent

In [6]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [7]:
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 14 documents to knowledge base

In [8]:
# Create and use the agent
rag_agent = Agent(model=groq_model,
              name="RAG Agent",
              role="Responds to questions about Thai recipes", 
              knowledge=knowledge_base,  

              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
              search_knowledge=False,
              show_tool_calls=True,
              markdown=False)


In [9]:
rag_agent.run("How do I make chicken and galangal in coconut milk soup")

RunResponse(content="Khao Soi's more known cousin, Massaman chicken is instead similar to a style dish for this flavor profile. A more suitable dish would be Tom Kha Gai, however here's a similar recipe. The difference between the dishes you had in mind could be what base I'm using or how quickly I include ingredients, so the following dish uses lemongrass for a citrus flavor, so galangal brings depth instead. \n\nHere's a simple recipe for the Thai Coconut Soup dish:\n\nIngredients:\n1. 1 tablespoon of vegetable oil\n2. 2 stalks lemongrass, slightly bruised\n3. 3-4 cloves garlic, minced\n4. 2 inches of galangal, cut into thin slices\n5. 2 pounds boneless, skinless chicken breast or thighs, cut into small bite-sized pieces\n6. 2 cups of chicken broth\n7. 1 can (14 oz) whole coconut milk\n8. 1 cup of coconut milk or coconut cream (for added richness)\n9. 2 tablespoons fish sauce\n10. 1 tablespoon of lime juice\n11. 1 tablespoon sugar\n12. 1 tablespoon Thai red curry paste (adjust the am

### Multi-Agent Team

In [10]:
team_leader = Agent(
    name="Multi-Agent Team",
    role="Team leader agent",
    model=groq_model,
    tools=[web_searcher,rag_agent],
    show_tool_calls=True,
    instructions=[
			"First think about the tools you have and use them to answer the question if needed.",
			"Then use the tools to answer the question.",
			"Reponse in markdown format or plain text.",
			"If you have used a tool, please include the tool name and description in the markdown format.",
		],
    markdown=False,
    debug_mode=False,
)


In [11]:
response = team_leader.run("Who is Thomas Edison?")
print(response.content)
print(response.messages)
print(response.metrics)

### Thomas Edison
Thomas Edison was a renowned American inventor, entrepreneur, and engineer who developed and commercialized numerous groundbreaking technologies. He is widely recognized as one of the most prolific inventors in history.

### Early Life and Education
Born on February 11, 1847, in Milan, Ohio, Edison moved to Port Huron, Michigan, at a young age. He was home-schooled and developed a passion for reading and experimenting with electrical systems. However, he struggled in school and was expelled from a boarding school in 1863.

### Career
Edison began working as a telegraph operator at the age of 16. He later developed the first stock ticker, the quadruplex telegraph, and the phonograph, which could record and play sound. Throughout his career, he held over 1,093 patents in his name, representing a wide range of inventions, including:

* *Light Bulb (1879): Edison developed the first commercially practical incandescent light bulb.* 
[Tool used: Wikipedia]
[Wikipedia](https

In [13]:
response =  team_leader.run("How do I make chicken and galangal in coconut milk soup")
print(response.content)
print(response.messages)
print(response.metrics)

**Recipe: Chicken and Galangal in Coconut Milk Soup**

**Description:** This is a traditional Southeast Asian-inspired soup that combines the richness of coconut milk with the aromatic flavor of galangal and the tenderness of chicken.

**Tools Used:**

* [Wikipedia](https://en.wikipedia.org/wiki/Reference_desk) - a free online encyclopedia that provides information on various topics, including recipes.
* [Allrecipes](https://www.allrecipes.com/) - a website that allows users to find and share recipes.

**Ingredients:**

* 2 lbs boneless, skinless chicken breast or thighs
* 2 inches of galangal, peeled and sliced
* 2 kaffir lime leaves
* 2 cloves of garlic, minced
* 1 tablespoon grated fresh ginger
* 2 cups of coconut milk
* 2 cups of chicken broth
* 1 tablespoon of fish sauce (optional)
* 1 tablespoon of lime juice
* Salt and pepper to taste
* Fresh cilantro or basil leaves for garnish

**Instructions:**

1. **Heat oil in a pan**: Heat 2 tablespoons of oil in a large pan over medium he